In [1]:
# Import Requests and xml 
import requests
import xml.etree.ElementTree as ET
import pandas as pd


In [10]:
browser = webdriver.Chrome(ChromeDriverManager().install())

# URL of the Senate roll call vote XML page
senate_bills_data = []
# 10 most recent roll call votes, will automate this later
vote_tallies = ["00255","00247","00230","00162","00161", "00160", "00158", "00157", "00154", "00153"]
for x in range(10):
    url = f'https://www.senate.gov/legislative/LIS/roll_call_votes/vote1182/vote_118_2_{vote_tallies[x]}.xml'

# Make a GET request to fetch the XML data
    response = requests.get(url)
# Parse the XML content    
    root = ET.fromstring(response.content)

# Find the Bill Number
    hr_number = root.find('document/document_name').text
    
#Extract Member info
    for member in root.find('members').findall('member'):
        first_name = member.find('first_name').text
        last_name = member.find('last_name').text
        party = member.find('party').text
        state = member.find('state').text
        vote_cast = member.find('vote_cast').text
        senate_bills_data.append({
            'Bill Number': hr_number,
            'First Name': first_name,
            'Last Name': last_name,
            'Party': party,
            'State': state,
            'Vote Cast': vote_cast
        })
# Print Variable to ensure data was stored properly
senate_bills_data_df = pd.DataFrame(senate_bills_data)
senate_bills_data_df


,Bill Number,First Name,Last Name,Party,State,Vote Cast
0,H.R. 9747,Tammy,Baldwin,D,WI,Yea
1,H.R. 9747,John,Barrasso,R,WY,Yea
2,H.R. 9747,Michael,Bennet,D,CO,Yea
3,H.R. 9747,Marsha,Blackburn,R,TN,Nay
4,H.R. 9747,Richard,Blumenthal,D,CT,Yea
...,...,...,...,...,...,...
995,H.R. 815,Peter,Welch,D,VT,Yea
996,H.R. 815,Sheldon,Whitehouse,D,RI,Yea
997,H.R. 815,Roger,Wicker,R,MS,Yea
998,H.R. 815,Ron,Wyden,D,OR,Yea


In [11]:
browser.quit()

In [17]:
#Export Senate Vote Tallies to CSV
senate_bills_data_df.to_csv('senate_vote_tallies.csv')

In [2]:
# Import Splinter and BeautifulSoup
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [30]:
browser = webdriver.Chrome(ChromeDriverManager().install())

# Empty List for data
house_bills_data = []

# 10 most recent votes, will automate this later
house_tallies = ["455","454","453","452","451", "450", "449", "448", "447", "446"]

#Cycle through the website for each vote and get the data for each
for x in range(10):
    url = f'https://clerk.house.gov/Votes/2024{house_tallies[x]}'

# Make a GET request to fetch the HTML data
    browser.get(url)

    html = browser.page_source
  
# Create a Beautiful Soup Object    
    soup = BeautifulSoup(html, 'html.parser')
    
# Extract all the text elements
    text_elements = soup.find_all('div', class_= "col-md-12 col-sm-12", id="legislative-main-content")
   
# Find the Bill Number
# Find the span that contains the bill number
    bill_number_span = soup.find('span', class_='legisNum')

# Extract the bill number text
    bill_number = bill_number_span.get_text(strip=True)


# Loop through the text elements
# Extract the Bill Number, Rep Name, Poltical Party, and State from the elements
# Store each Bill Number, Rep Name, Poltical Party, and State in a dictionary
# Add the dictionary to the list

    rows = soup.find('tbody', {'id': 'member-votes'}).find_all('tr')
 

    for row in rows:
   # Safely extract the representative name
        representative_tag = row.find('a')
        representative = representative_tag.get_text(strip=True) if representative_tag else 'N/A'
    
    # Safely extract the party
        party_tag = row.find('td', {'data-label': 'party'})
        party = party_tag.get_text(strip=True) if party_tag else 'N/A'
    
    # Safely extract the state
        state_tag = row.find('td', {'data-label': 'state', 'class': 'hidden-sm hidden-xs'})
        state = state_tag.get_text(strip=True) if state_tag else 'N/A'
    
    # Safely extract the vote
        vote_tag = row.find('td', {'data-label': 'vote'})
        vote = vote_tag.get_text(strip=True) if vote_tag else 'N/A'

        
    # Append the extracted data to the votes list
        house_bills_data.append({
            'Bill Number':bill_number,
            'Representative': representative,
            'Party': party,
            'State': state,
            'Vote': vote
        })
#place all vote data into a dataframe
house_bills_data_df = pd.DataFrame(house_bills_data)
house_bills_data_df
print(type(house_bills_data_df))


<class 'pandas.core.frame.DataFrame'>


In [26]:
browser.quit()

In [22]:
# get first and last names of 1st house rep
browser = webdriver.Chrome(ChromeDriverManager().install())

# Empty List for data
house_links_data = []

#Cycle through the website for each vote and get the data for each
url = f'https://clerk.house.gov/Votes/2024455'

# Make a GET request to fetch the HTML data
browser.get(url)

html = browser.page_source
  
# Create a Beautiful Soup Object    
soup = BeautifulSoup(html, 'html.parser')
    
# Extract all the text elements
house_elements = soup.find_all('div', class_= "col-md-12 col-sm-12", id="legislative-main-content")
representative_rows = soup.find_all('a', class_='library-link')


# Find the all reps link
for rep in representative_rows:
        # Extract the representative's name
        rep_name = rep.text.strip()
        
        # Extract the link (href attribute)
        rep_link = rep['href']
        
        # If the link is relative, add the base URL
        full_link = 'https://clerk.house.gov' + rep_link

        rep_response = requests.get(full_link)

        if rep_response.status_code == 200:
       # Parse Reps' profile page
            rep_soup = BeautifulSoup(rep_response.content, 'html.parser')
        
    # Assuming the full name is in a specific tag like <h1>, find it
            full_name_tag = rep_soup.find('h1', class_='library-h1')
            full_name = full_name_tag.text.strip()

    # Extract Rep Phone Number
            phone_tag = rep_soup.find('span', class_='phone')
            phone_number = phone_tag.text.strip().replace('Phone:','') 
        #Add rep link data to dictionary
        house_links_data.append({
            'Name':rep_name,
            'link': full_link,
            'Full Name':full_name,
            'Phone Number':phone_number
            })
house_links_data_df = pd.DataFrame(house_links_data)
house_links_data_df

,Name,link,Full Name,Phone Number
0,Adams,https://clerk.house.gov/Members/A000370,Alma S. Adams,(202) 225-1510
1,Aderholt,https://clerk.house.gov/Members/A000055,Robert B. Aderholt,(202) 225-4876
2,Aguilar,https://clerk.house.gov/Members/A000371,Pete Aguilar,(202) 225-3201
3,Alford,https://clerk.house.gov/Members/A000379,Mark Alford,(202) 225-2876
4,Allen,https://clerk.house.gov/Members/A000372,Rick W. Allen,(202) 225-2823
...,...,...,...,...
427,Wilson (SC),https://clerk.house.gov/Members/W000795,Joe Wilson,(202) 225-2452
428,Wittman,https://clerk.house.gov/Members/W000804,Robert J. Wittman,(202) 225-4261
429,Womack,https://clerk.house.gov/Members/W000809,Steve Womack,(202) 225-4301
430,Yakym,https://clerk.house.gov/Members/Y000067,Rudy Yakym III,(202) 225-3915


In [31]:
print(type(house_bills_data_df))

<class 'pandas.core.frame.DataFrame'>


In [36]:
# Remove N/A from DF
house_bills_data_df = house_bills_data_df.dropna()

#merge House links and House Bills Data
house_bills_data_df_clean = pd.merge(house_bills_data_df,house_links_data_df, left_on='Representative', right_on = 'Name')
house_bills_data_df_clean = house_bills_data_df_clean.drop(columns=['Name'])
house_bills_data_df_clean_sorted = house_bills_data_df_clean.sort_values(by='Bill Number')
house_bills_data_df_clean_sorted

,Bill Number,Representative,Party,State,Vote,link,Full Name,Phone Number
1963,H. R. 3334,Johnson (GA),Democratic,Georgia,Nay,https://clerk.house.gov/Members/J000288,"Henry C. ""Hank"" Johnson, Jr.",(202) 225-1605
1304,H. R. 3334,Fong,Republican,California,Aye,https://clerk.house.gov/Members/F000480,Vince Fong,(202) 225-2915
3534,H. R. 3334,Scott (VA),Democratic,Virginia,Aye,https://clerk.house.gov/Members/S000185,"Robert C. ""Bobby"" Scott",(202) 225-8351
653,H. R. 3334,Cherfilus-McCormick,Democratic,Florida,Nay,https://clerk.house.gov/Members/C001127,Sheila Cherfilus-McCormick,(202) 225-1313
654,H. R. 3334,Cherfilus-McCormick,Democratic,Florida,Aye,https://clerk.house.gov/Members/C001127,Sheila Cherfilus-McCormick,(202) 225-1313
...,...,...,...,...,...,...,...,...
620,H. Res. 1469,Castor (FL),Democratic,Florida,Nay,https://clerk.house.gov/Members/C001066,Kathy Castor,(202) 225-3376
3580,H. Res. 1469,Sewell,Democratic,Alabama,Nay,https://clerk.house.gov/Members/S001185,Terri A. Sewell,(202) 225-2665
1660,H. Res. 1469,Guest,Republican,Mississippi,Yea,https://clerk.house.gov/Members/G000591,Michael Guest,(202) 225-5031
2340,H. Res. 1469,Lee (PA),Democratic,Pennsylvania,Nay,https://clerk.house.gov/Members/L000602,Summer L. Lee,(202) 225-2135


In [37]:
#Export House Vote Tallies to CSV
house_bills_data_df_clean_sorted.to_csv('house_vote_tallies.csv')